In [58]:
val rawblocks = sc.textFile("file:///var/lib/myspark/linkage/csvdata")
rawblocks.first

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"

In [59]:
rawblocks.count

5749142

In [60]:
val head = rawblocks.take(10)
head.length

10

In [61]:
head.foreach(println)

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"
37671,78628,1,?,1,?,1,1,1,1,1,TRUE
57064,74984,1,1,1,?,1,1,1,1,1,TRUE
3149,8328,1,?,1,?,1,1,1,1,1,TRUE
33771,38173,1,?,1,?,1,1,1,1,1,TRUE
17029,37657,1,?,1,?,1,1,1,1,1,TRUE
94492,94494,1,?,1,?,1,1,1,1,1,TRUE
2318,14303,1,?,1,?,1,1,1,1,1,TRUE
2481,10949,1,?,1,?,1,1,1,1,1,TRUE
25247,25711,1,?,1,?,1,1,1,1,1,TRUE


In [62]:
def isHeader(line:String) = line.contains("id_1")
head.filter(isHeader).foreach(println)

"id_1","id_2","cmp_fname_c1","cmp_fname_c2","cmp_lname_c1","cmp_lname_c2","cmp_sex","cmp_bd","cmp_bm","cmp_by","cmp_plz","is_match"


In [63]:
head.filter(!isHeader(_)).foreach(println)

37671,78628,1,?,1,?,1,1,1,1,1,TRUE
57064,74984,1,1,1,?,1,1,1,1,1,TRUE
3149,8328,1,?,1,?,1,1,1,1,1,TRUE
33771,38173,1,?,1,?,1,1,1,1,1,TRUE
17029,37657,1,?,1,?,1,1,1,1,1,TRUE
94492,94494,1,?,1,?,1,1,1,1,1,TRUE
2318,14303,1,?,1,?,1,1,1,1,1,TRUE
2481,10949,1,?,1,?,1,1,1,1,1,TRUE
25247,25711,1,?,1,?,1,1,1,1,1,TRUE


In [64]:
val noheader = rawblocks.filter(!isHeader(_))
noheader.first
noheader.count

5749132

In [65]:
noheader.filter(x=>x.contains("cmp_fname_c1")).collect

Array()

In [66]:
val line = head(5)
val pieces = line.split(',')
pieces.foreach(println)

17029
37657
1
?
1
?
1
1
1
1
1
TRUE


In [67]:
val id1 = pieces(0).toInt
val id2 = pieces(1).toInt
val matched = pieces(11).toBoolean
println(id1+" "+id2+" "+matched)

17029 37657 true


In [68]:
val rawscores = pieces.slice(2,11)
rawscores.map(s => s.toDouble)

Name: java.lang.NumberFormatException
Message: For input string: "?"
StackTrace:   at sun.misc.FloatingDecimal.readJavaFormatString(FloatingDecimal.java:2043)
  at sun.misc.FloatingDecimal.parseDouble(FloatingDecimal.java:110)
  at java.lang.Double.parseDouble(Double.java:538)
  at scala.collection.immutable.StringLike$class.toDouble(StringLike.scala:284)
  at scala.collection.immutable.StringOps.toDouble(StringOps.scala:29)
  at $anonfun$1.apply(<console>:42)
  at $anonfun$1.apply(<console>:42)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
  at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
  at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
  at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:186)

In [69]:
def toDouble(s:String)={
    if("?".equals(s)) Double.NaN else s.toDouble
}
val scores = rawscores.map(toDouble)
scores.foreach(println)

1.0
NaN
1.0
NaN
1.0
1.0
1.0
1.0
1.0


In [70]:
def parseOld(line:String)={
    val pieces = line.split(",")
    val id1 = pieces(0).toInt
    val id2 = pieces(1).toInt
    val scores = pieces.slice(2,11).map(toDouble)
    val matched = pieces(11).toBoolean
    (id1, id2, scores, matched)
}
val tup = parseOld(line)
tup.toString

(17029,37657,[D@6a2cb182,true)

In [71]:
case class MatchData(id1:Int, id2:Int,
                    scores:Array[Double], matched: Boolean)
def parse(line:String)={
    val pieces = line.split(",")
    val id1 = pieces(0).toInt
    val id2 = pieces(1).toInt
    val scores = pieces.slice(2,11).map(toDouble)
    val matched = pieces(11).toBoolean
    MatchData(id1, id2, scores, matched)
}
val md = parse(line)
println(md.id1+" "+md.id2+" "+md.scores.length+" "+md.matched)

17029 37657 9 true


In [72]:
val mds = head.filter(!isHeader(_)).map(parse(_))
mds.head
mds.length

9

In [73]:
val parsed = noheader.map(parse(_))
parsed.cache
parsed.count

5749132

In [74]:
val grouped = mds.groupBy(md=>md.matched)
grouped.mapValues(x=>x.size).foreach(println)

(true,9)


In [75]:
//val allgrouped = parsed.groupBy(x=>x.matched)
//allgrouped.mapValues(_.size).foreach(println)

Name: Syntax Error.
Message: 
StackTrace: 

In [76]:
val matchCount = parsed.map(x=>x.matched).countByValue()
matchCount.size

2

In [77]:
matchCount.foreach(println)

(true,20931)
(false,5728201)


In [78]:
val matchCountSeq = matchCount.toSeq

In [79]:
matchCountSeq.sortBy(_._1).foreach(println)

(false,5728201)
(true,20931)


In [80]:
matchCountSeq.sortBy(_._2).foreach(println)

(true,20931)
(false,5728201)


In [81]:
matchCountSeq.sortBy(_._2).reverse.foreach(println)

(false,5728201)
(true,20931)


In [82]:
parsed.map(x=>x.scores(0)).stats()

(count: 5749132, mean: NaN, stdev: NaN, max: NaN, min: NaN)

In [83]:
import java.lang.Double.isNaN
parsed.map(x=>x.scores(0)).filter(!isNaN(_)).stats()

(count: 5748125, mean: 0.712902, stdev: 0.388758, max: 1.000000, min: 0.000000)

In [84]:
val stats = (0 until 9).map(i=>{
    println("stat[" + i + "]:" +parsed.map(x=>x.scores(i)).filter(!isNaN(_)).stats())
})

stat[0]:(count: 5748125, mean: 0.712902, stdev: 0.388758, max: 1.000000, min: 0.000000)
stat[1]:(count: 103698, mean: 0.900018, stdev: 0.271316, max: 1.000000, min: 0.000000)
stat[2]:(count: 5749132, mean: 0.315628, stdev: 0.334234, max: 1.000000, min: 0.000000)
stat[3]:(count: 2464, mean: 0.318413, stdev: 0.368492, max: 1.000000, min: 0.000000)
stat[4]:(count: 5749132, mean: 0.955001, stdev: 0.207301, max: 1.000000, min: 0.000000)
stat[5]:(count: 5748337, mean: 0.224465, stdev: 0.417230, max: 1.000000, min: 0.000000)
stat[6]:(count: 5748337, mean: 0.488855, stdev: 0.499876, max: 1.000000, min: 0.000000)
stat[7]:(count: 5748337, mean: 0.222749, stdev: 0.416091, max: 1.000000, min: 0.000000)
stat[8]:(count: 5736289, mean: 0.005529, stdev: 0.074149, max: 1.000000, min: 0.000000)


In [85]:
//import org.apache.spark.util.StatCounter
class NAStatCounter extends Serializable{
    import org.apache.spark.util.StatCounter
    val stats = new StatCounter
    var missing: Long = 0

    def add(x:Double):NAStatCounter ={
        if(java.lang.Double.isNaN(x)){
            missing += 1
        }else{
            stats.merge(x)
        }
        this
    }
    
    def merge(other: NAStatCounter): NAStatCounter = {
        stats.merge(other.stats)
        missing += other.missing
        this
    }
    
    override def toString = {
        "stats: " + stats.toString + " NaN: " + missing
    }
}

object NAStatCounter extends Serializable{
    def apply(x: Double) = new NAStatCounter().add(x)
}

val nastats = NAStatCounter(17.29)
nastats.toString

stats: (count: 1, mean: 17.290000, stdev: 0.000000, max: 17.290000, min: 17.290000) NaN: 0

In [86]:
val nas1 = NAStatCounter(10.0)
nas1.add(2.1)
nas1.toString

stats: (count: 2, mean: 6.050000, stdev: 3.950000, max: 10.000000, min: 2.100000) NaN: 0

In [87]:
val nas2 = NAStatCounter(Double.NaN)
nas1.merge(nas2)
nas1.toString

stats: (count: 2, mean: 6.050000, stdev: 3.950000, max: 10.000000, min: 2.100000) NaN: 1

In [88]:
val arr = Array(1.0, Double.NaN, 17.29)
val nas = arr.map(d => NAStatCounter(d))
nas.foreach(e=>println(e.toString))

stats: (count: 1, mean: 1.000000, stdev: 0.000000, max: 1.000000, min: 1.000000) NaN: 0
stats: (count: 0, mean: 0.000000, stdev: NaN, max: -Infinity, min: Infinity) NaN: 1
stats: (count: 1, mean: 17.290000, stdev: 0.000000, max: 17.290000, min: 17.290000) NaN: 0


In [89]:
val nas1 = Array(1.0, Double.NaN).map(NAStatCounter(_))
val nas2 = Array(Double.NaN, 2.0).map(NAStatCounter(_))
val merged = nas1.zip(nas2).map(p=> p._1.merge(p._2))
merged.foreach(e=>println(e.toString))

stats: (count: 1, mean: 1.000000, stdev: 0.000000, max: 1.000000, min: 1.000000) NaN: 1
stats: (count: 1, mean: 2.000000, stdev: 0.000000, max: 2.000000, min: 2.000000) NaN: 1


In [90]:
val nas11 = Array(1.0, Double.NaN).map(NAStatCounter(_))
val nas12 = Array(Double.NaN, 2.0).map(NAStatCounter(_))
val merged1 = nas11.zip(nas12).map{case(a,b)=>a.merge(b)}
merged1.foreach(e=>println(e.toString))

stats: (count: 1, mean: 1.000000, stdev: 0.000000, max: 1.000000, min: 1.000000) NaN: 1
stats: (count: 1, mean: 2.000000, stdev: 0.000000, max: 2.000000, min: 2.000000) NaN: 1


In [91]:
val nas21 = Array(1.0, Double.NaN).map(NAStatCounter(_))
val nas22 = Array(Double.NaN, 2.0).map(NAStatCounter(_))
val nas = List(nas21, nas22)
val merged2 = nas.reduce((n1,n2)=>{
    n1.zip(n2).map{case(a,b)=>a.merge(b)}
})
merged2.foreach(e=>println(e.toString))

stats: (count: 1, mean: 1.000000, stdev: 0.000000, max: 1.000000, min: 1.000000) NaN: 1
stats: (count: 1, mean: 2.000000, stdev: 0.000000, max: 2.000000, min: 2.000000) NaN: 1


In [92]:
val nasRDD = parsed.map(md => {
    md.scores.map(NAStatCounter(_))
})
val reduced = nasRDD.reduce((n1,n2)=>{
    n1.zip(n2).map{case(a,b)=>a.merge(b)}
})
reduced.foreach(println)

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 364, localhost, executor driver): java.lang.NoClassDefFoundError: Could not initialize class $line89.$read$
	at $line360.$read$$iw.<init>(<console>:13)
	at $line360.$read.<init>(<console>:79)
	at $line360.$read$.<init>(<console>:83)
	at $line360.$read$.<clinit>(<console>)
	at $line361.$read$$iw$$iw$$iw$$iw$$iw$$iw$NAStatCounter$.apply(<console>:67)
	at $line393.$read$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1$$anonfun$apply$1.apply(<console>:70)
	at $line393.$read$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1$$anonfun$apply$1.apply(<console>:70)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$of